
```mermaid
Input: agent_state, cls_state, global_state, edges
       |
       v
+------------------+     +-------------------+
| AgentEncoder     |     | ClsEncoder        |
| (fc_block)       |     | (GCN -> Mean Pool)|
+------------------+     +-------------------+
       |                       |
       v                       v
agent_emb                  cls_emb
       |                       |
       v                       v
+------------------+     +-------------------+
| SelfAttention   |     | GraphWeight (GCN)  |
| (Self-Attention)|     | (GCN -> Attention) |
+------------------+     +-------------------+
       |                       |
       v                       v
integrate_obs_emb          integrate_cls_emb
       |                       |
       v                       v
+----------------------------------+
| Add (integrate_obs_emb + integrate_cls_emb) |
+----------------------------------+
       |
       v
critic_head (MLP -> RegressionHead)
       |
       v
Output: value
```

### 总结

- **GCN** 是一个包含两层图卷积的网络，用于处理图结构数据。
- **ClsEncoder** 使用 GCN 对邻域状态进行编码，并通过均值池化得到最终的编码结果。
- **GraphWeight** 使用两个 GCN 分别生成查询和键，然后通过注意力机制计算节点之间的关系权重。
- **整体结构** 包含了多个模块的协同工作，最终通过 Critic Head 计算 Q 值输出。

如果你有具体的绘图工具或需求，可以进一步细化这些结构图。

# 交叉注意力CrosAttention

In [3]:
import torch
import torch.nn.functional as F
import math
from torch import nn
import unittest

class CrosAttention(nn.Module):
    def __init__(self, input_dim=4, hidden_dim=4):
        super(CrosAttention, self).__init__()
        self._hidden_dim = hidden_dim
        # 输入和输出的维度，点乘
        self.query = nn.Linear(input_dim, hidden_dim)
        self.key = nn.Linear(input_dim, hidden_dim)
        self.value = nn.Linear(input_dim, hidden_dim)

    def forward(self, agent_state, integrate_cluster_emb):
        T, B, A, N = agent_state.shape

        # Cross-attention
        query = self.query(integrate_cluster_emb)       # integrate_cluster_emb作为query
        key = self.key(agent_state)                     # agent_state作为key
        value = self.value(agent_state)                 # agent_state作为value
        # 矩阵点积
        attention_map = torch.matmul(query, key.transpose(-1, -2))  # T, B, 1, A
        attention_map /= math.sqrt(self._hidden_dim)
        attention_map = F.softmax(attention_map, dim=-1)
        # 输出注意力权重，attention_map点乘value向量
        cross_attention_output = torch.matmul(attention_map, value)  # T, B, 1, hidden_dim
        
        return cross_attention_output  # T, B, 1, hidden_dim


## 参数学习机制
### 线性层 (nn.Linear):
- self.query = nn.Linear(input_dim, hidden_dim): 这是一个线性层，将输入维度 input_dim 映射到隐藏维度 hidden_dim。它包含两个可学习的参数：权重矩阵和偏置向量。
- self.key = nn.Linear(input_dim, hidden_dim): 同样是一个线性层，将输入维度 input_dim 映射到隐藏维度 hidden_dim。它也包含权重矩阵和偏置向量。
- self.value = nn.Linear(input_dim, hidden_dim): 同样是一个线性层，将输入维度 input_dim 映射到隐藏维度 hidden_dim。它也包含权重矩阵和偏置向量。
### 前向传播 (forward 方法):
- query = self.query(integrate_cluster_emb): 使用 query 线性层将 integrate_cluster_emb 映射到隐藏维度 hidden_dim。
- key = self.key(agent_state): 使用 key 线性层将 agent_state 映射到隐藏维度 hidden_dim。
- value = self.value(agent_state): 使用 value 线性层将 agent_state 映射到隐藏维度 hidden_dim。
- attention_map = torch.matmul(query, key.transpose(-1, -2)): 计算查询和键之间的注意力分数。
- attention_map /= math.sqrt(self._hidden_dim): 对注意力分数进行缩放。
- attention_map = F.softmax(attention_map, dim=-1): 对注意力分数进行 softmax 归一化。
- cross_attention_output = torch.matmul(attention_map, value): 使用注意力权重对值进行加权求和。
### 参数学习过程
在训练过程中，优化器（如 Adam 或 SGD）会更新这些线性层的权重和偏置，以最小化损失函数。具体来说，优化器会根据反向传播计算出的梯度来调整这些参数，从而使得模型能够更好地学习到数据中的模式。

In [ ]:

# 创建 CrosAttention 实例
cros_attention = CrosAttention(input_dim=4, hidden_dim=4)

# 查看参数
for name, param in cros_attention.named_parameters():
    print(f"Parameter name: {name}, Shape: {param.shape}, Requires Grad: {param.requires_grad}")

  - 前向传播过程：
    - 获取输入 `agent_state` 的形状 `(T, B, A, N)`，其中：
      - `T`: 时间步数。
      - `B`: 批次大小。
      - `A`: 智能体数量。
      - `N`: 每个智能体的特征维度。

In [41]:
class TestCrosAttention(unittest.TestCase):
    def setUp(self):
        self.input_dim = 4
        self.hidden_dim = 4
        self.model = CrosAttention(input_dim=self.input_dim, hidden_dim=self.hidden_dim)

    def test_forward_output_shape(self):
        T, B, A, N = 2, 3, 5, self.input_dim
        agent_state = torch.randn(T, B, A, N)

        integrate_cluster_emb = torch.randn(T, B, 1, N)
        print("agent_state shape is ", agent_state.shape)
        # print("agent_state is \n", agent_state)
        print("integrate_cluster_emb shape is ", integrate_cluster_emb.shape)
        # print("integrate_cluster_emb is \n", integrate_cluster_emb)
        output = self.model(agent_state, integrate_cluster_emb)
        print("output shape is ", output.shape)
        # print("output is \n", output)
        self.assertEqual(output.shape, (T, B, 1, self.hidden_dim))

    def test_forward_attention_behavior(self):
        T, B, A, N = 2, 3, 5, self.input_dim
        agent_state = torch.randn(T, B, A, N)
        integrate_cluster_emb = torch.randn(T, B, 1, N)
        
        output = self.model(agent_state, integrate_cluster_emb)

        print("model output is \n", output)
        # 验证输出是否在预期范围内
        self.assertTrue(torch.all(output >= 0))
        self.assertTrue(torch.all(output <= 1))
        
        # 验证输出的平均值是否接近0.5（假设均匀分布）
        self.assertTrue(torch.allclose(output.mean(dim=-1), torch.tensor(0.5), atol=0.1))

In [ ]:
cross_attention_test = TestCrosAttention()
cross_attention_test.setUp()
# cross_attention_test.test_forward_attention_behavior()

# 自注意力 SelfAttention

In [6]:

class SelfAttention(nn.Module):
    def __init__(self, input_dim=4, hidden_dim=4):
        super(SelfAttention, self).__init__()
        self._hidden_dim = hidden_dim
        self.query = nn.Linear(input_dim, hidden_dim)
        self.key = nn.Linear(input_dim, hidden_dim)
        self.value = nn.Linear(input_dim, hidden_dim)

    def forward(self, agent_state):
        T, B, A, N = agent_state.shape
        # print("agent_state shape is ", agent_state.shape)
        query = self.query(agent_state)
        key = self.key(agent_state)
        value = self.value(agent_state)
        
        attention_map = torch.matmul(query, key.transpose(-1, -2))  # T, B, A, A
        attention_map /= math.sqrt(self._hidden_dim)
        attention_map = F.softmax(attention_map, dim=-1)
        self_attention_output = torch.matmul(attention_map, value)  # T, B, A, hidden_dim
        
        return self_attention_output  # T, B, A, hidden_dim


class TestSelfAttention(unittest.TestCase):
    def setUp(self):
        self.input_dim = 4
        self.hidden_dim = 4
        self.model = SelfAttention(input_dim=self.input_dim, hidden_dim=self.hidden_dim)

    def test_forward_output_shape(self):
        T, B, A = 3, 2, 5
        agent_state = torch.randn(T, B, A, self.input_dim)
        output = self.model(agent_state)
        print("output shape is ", output.shape)
        self.assertEqual(output.shape, (T, B, A, self.hidden_dim))

    def test_forward_output_range(self):
        T, B, A = 3, 2, 5
        agent_state = torch.randn(T, B, A, self.input_dim)
        output = self.model(agent_state)
        print("output is ", output)
        self.assertTrue(output.max() <= 1.0)
        self.assertTrue(output.min() >= -1.0)

    def test_forward_different_hidden_dim(self):
        self.hidden_dim = 8
        self.model = SelfAttention(input_dim=self.input_dim, hidden_dim=self.hidden_dim)
        T, B, A = 3, 2, 5
        agent_state = torch.randn(T, B, A, self.input_dim)
        output = self.model(agent_state)
        self.assertEqual(output.shape, (T, B, A, self.hidden_dim))


In [ ]:
self_attention_test = TestSelfAttention()
self_attention_test.setUp()
self_attention_test.test_forward_output_shape()
self_attention_test.test_forward_output_range()

单元测试设计
被测方法行为：

- AgentEncoder 类是一个继承自 nn.Module 的 PyTorch 模块，它包含一个编码器，该编码器是一个全连接块（fc_block），用于将输入的 agent_state 转换为隐藏表示。
- ClsEncoder 类也是一个继承自 nn.Module 的 PyTorch 模块，它包含一个编码器，该编码器是一个图卷积网络（GCN），用于将 neigborhood_state 和 edges 转换为隐藏表示。然后，它通过在倒数第二个维度上取平均值来聚合这些表示。

# ClsEncoder簇编码

In [ ]:
import networkx as nx
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from IPython.display import HTML

# 创建图
G = nx.Graph()
G.add_nodes_from(range(5))
G.add_edges_from([(0, 1), (1, 2), (2, 3), (3, 4), (4, 0)])

# 定义图的布局
pos = nx.spring_layout(G, seed=42)

# 创建图形对象
fig, ax = plt.subplots()

# 定义动画函数
def update(num, G, pos, ax):
    ax.clear()
    nx.draw(G, pos, with_labels=True, node_color='lightblue', node_size=500, edge_color='gray', width=2, ax=ax)
    ax.set_title(f'Frame {num}')

# 创建动画
ani = animation.FuncAnimation(fig, update, frames=range(10), fargs=(G, pos, ax), repeat=False)

# 将动画转换为 HTML
html_animation = ani.to_jshtml()

# 在 Jupyter Notebook 中显示动画

# HTML(html_animation)


In [9]:
import torch
import torch.nn as nn
from torch_geometric.nn import GCNConv
from torch_geometric.data import Data
import unittest

class GCN(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super(GCN, self).__init__()
        self.conv1 = GCNConv(input_dim, hidden_dim)
        self.conv2 = GCNConv(hidden_dim, output_dim)

    def forward(self, x, edge_index):
        x = self.conv1(x, edge_index)
        x = F.relu(x)
        x = self.conv2(x, edge_index)
        return x

class ClsEncoder(nn.Module):
    def __init__(self, input_dim, hidden_dim: int = 4, output_dim: int = 4):
        super(ClsEncoder, self).__init__()
        self._hidden_dim = hidden_dim
        self.encoder = GCN(input_dim, hidden_dim, output_dim)

    def forward(self, neigborhood_state, edges):
        neighborhood_emb = self.encoder(neigborhood_state, edges)
        neighborhood_emb = torch.mean(neighborhood_emb, dim=-2, keepdim=False)
        return neighborhood_emb


In [ ]:
import torch
import unittest

class TestClsEncoder(unittest.TestCase):
    def setUp(self):
        # 初始化测试参数
        self.input_dim = 3
        self.hidden_dim = 4
        self.output_dim = 2
        self.num_nodes = 5
        
        # 创建模型实例
        self.model = ClsEncoder(
            input_dim=self.input_dim,
            hidden_dim=self.hidden_dim,
            output_dim=self.output_dim
        )
        
        # 创建测试用图数据
        self.neighborhood_state = torch.randn(self.num_nodes, self.input_dim)
        self.edge_index = torch.tensor([[0,1,2,3,4], [1,2,3,4,0]], dtype=torch.long)

    def test_output_shape(self):
        """测试输出维度是否正确"""
        output = self.model(self.neighborhood_state, self.edge_index)
        print("output shape:", output.shape)
        self.assertEqual(output.shape, (self.output_dim,))

    def test_mean_aggregation(self):
        """测试均值聚合是否正确"""
        # 冻结网络参数
        with torch.no_grad():
            # 将第一层GCN参数置零，bias置1
            self.model.encoder.conv1.lin.weight.zero_()
            self.model.encoder.conv1.bias.fill_(1.0)
            
            # 将第二层GCN参数置零，bias置2
            self.model.encoder.conv2.lin.weight.zero_()
            self.model.encoder.conv2.bias.fill_(2.0)

        # 创建全零输入特征
        test_input = torch.zeros(self.num_nodes, self.input_dim)
        
        # 前向传播
        output = self.model(test_input, self.edge_index)
        print("output shape:", output.shape)
        # 验证输出值全为2.0（ReLU后的bias传播）
        expected = torch.full((self.output_dim,), 2.0)
        self.assertTrue(torch.allclose(output, expected, atol=1e-6))

# if __name__ == "__main__":
#     unittest.main()
if __name__ == '__main__':
    unittest.main(argv=['first-arg-is-ignored'], exit=False)

In [ ]:
cls_encoder_test = TestClsEncoder()
cls_encoder_test.setUp()
cls_encoder_test.test_mean_aggregation()
# cls_encoder_test.test_output_shape()

# fc_block 


 fc_block 函数用于创建一个包含线性层、可选的归一化层、可选的激活函数和可选的丢弃层的全连接块。
 该函数接受以下参数：

- in_channels: 输入张量的通道数。
- out_channels: 输出张量的通道数。
- activation: 一个可选的激活函数。
- norm_type: 归一化类型，可以是 None 或指定的归一化类型。
- use_dropout: 一个布尔值，指示是否使用丢弃层。
- dropout_probability: 丢弃概率，默认为 0.5。
函数根据提供的参数构建一个 nn.Sequential 对象，其中包含线性层、可选的归一化层、可选的激活函数和可选的丢弃层。

分支和所需的测试用例:

- 线性层: 总是存在，因此不需要特定的测试用例。
- 归一化层: 如果 norm_type 不是 None，则包含一个归一化层。需要测试 norm_type 为 None 和非 None 的情况。
- 激活函数: 如果 activation 不是 None，则包含一个激活函数。需要测试 activation 为 None 和非 None 的情况。
- 丢弃层: 如果 use_dropout 为 True，则包含一个丢弃层。需要测试 use_dropout 为 True 和 False 的情况。
- 模拟需求: 不需要模拟，因为该函数不依赖于外部系统或需要模拟的复杂对象。它仅使用 PyTorch 的标准组件。

In [39]:
import sys

sys.path.append(r"C:\Users\Eddie\Documents\marl_sigctrl\framework")
sys.path.append(r"C:\Users\Eddie\Documents\marl_sigctrl")


In [ ]:
import torch
import torch.nn as nn
from ding.torch_utils.network.nn_module import fc_block


def test_fc_block_NoActivationNoNormNoDropout():
    block = fc_block(in_channels=10, out_channels=20)
    assert len(block) == 1
    assert isinstance(block[0], nn.Linear)
    assert block[0].in_features == 10
    assert block[0].out_features == 20


def test_fc_block_WithActivationNoNormNoDropout():
    block = fc_block(in_channels=10, out_channels=20, activation=nn.ReLU())
    assert len(block) == 2
    assert isinstance(block[0], nn.Linear)
    assert isinstance(block[1], nn.ReLU)


def test_fc_block_NoActivationWithNormNoDropout():
    block = fc_block(in_channels=10, out_channels=20, norm_type='LN')
    assert len(block) == 2
    assert isinstance(block[0], nn.Linear)
    assert isinstance(block[1], nn.LayerNorm)


def test_fc_block_WithActivationWithNormNoDropout():
    block = fc_block(in_channels=10, out_channels=20, activation=nn.ReLU(), norm_type='LN')
    assert len(block) == 3
    assert isinstance(block[0], nn.Linear)
    assert isinstance(block[1], nn.LayerNorm)
    assert isinstance(block[2], nn.ReLU)


def test_fc_block_NoActivationNoNormWithDropout():
    block = fc_block(in_channels=10, out_channels=20, use_dropout=True)
    assert len(block) == 2
    assert isinstance(block[0], nn.Linear)
    assert isinstance(block[1], nn.Dropout)
    assert block[1].p == 0.5


def test_fc_block_WithActivationNoNormWithDropout():
    block = fc_block(in_channels=10, out_channels=20, activation=nn.ReLU(), use_dropout=True)
    assert len(block) == 3
    assert isinstance(block[0], nn.Linear)
    assert isinstance(block[1], nn.ReLU)
    assert isinstance(block[2], nn.Dropout)
    assert block[2].p == 0.5


def test_fc_block_NoActivationWithNormWithDropout():
    block = fc_block(in_channels=10, out_channels=20, norm_type='LN', use_dropout=True)
    assert len(block) == 3
    assert isinstance(block[0], nn.Linear)
    assert isinstance(block[1], nn.LayerNorm)
    assert isinstance(block[2], nn.Dropout)
    assert block[2].p == 0.5


def test_fc_block_WithActivationWithNormWithDropout():
    block = fc_block(in_channels=10, out_channels=20, activation=nn.ReLU(), norm_type='LN', use_dropout=True)
    assert len(block) == 4
    assert isinstance(block[0], nn.Linear)
    assert isinstance(block[1], nn.LayerNorm)
    assert isinstance(block[2], nn.ReLU)
    assert isinstance(block[3], nn.Dropout)
    assert block[3].p == 0.5

In [5]:
test_fc_block_WithActivationNoNormWithDropout()
test_fc_block_WithActivationWithNormWithDropout()


# AgentEncoder

In [6]:
import torch
import torch.nn as nn
from ding.torch_utils.network.nn_module import fc_block
class AgentEncoder(nn.Module):
    def __init__(self, 
                 input_dim,
                 hidden_dim: int = 4
        ):
        super(AgentEncoder, self).__init__()
        # 如果没有额外的参数，就是一个线性层
        self.encoder = fc_block(input_dim, hidden_dim)
        return 

    def forward(self, agent_state):
        return self.encoder(agent_state)



In [7]:

def test_AgentEncoder_DefaultHiddenDim():
    input_dim = 10
    encoder = AgentEncoder(input_dim)
    assert isinstance(encoder.encoder, nn.Sequential)
    assert len(encoder.encoder) == 1
    assert isinstance(encoder.encoder[0], nn.Linear)
    assert encoder.encoder[0].in_features == input_dim
    assert encoder.encoder[0].out_features == 4  # 默认 hidden_dim 为 4

    # 测试 forward 方法
    input_tensor = torch.randn(1, input_dim)
    output_tensor = encoder(input_tensor)
    assert output_tensor.shape == (1, 4)


def test_AgentEncoder_CustomHiddenDim():
    input_dim = 10
    hidden_dim = 8
    encoder = AgentEncoder(input_dim, hidden_dim)
    assert isinstance(encoder.encoder, nn.Sequential)
    assert len(encoder.encoder) == 1
    assert isinstance(encoder.encoder[0], nn.Linear)
    assert encoder.encoder[0].in_features == input_dim
    assert encoder.encoder[0].out_features == hidden_dim

    # 测试 forward 方法
    input_tensor = torch.randn(1, input_dim)
    output_tensor = encoder(input_tensor)
    assert output_tensor.shape == (1, hidden_dim)


def test_AgentEncoder_DifferentInputDimensions():
    input_dims = [5, 10, 20]
    hidden_dim = 8
    for input_dim in input_dims:
        encoder = AgentEncoder(input_dim, hidden_dim)
        assert isinstance(encoder.encoder, nn.Sequential)
        assert len(encoder.encoder) == 1
        assert isinstance(encoder.encoder[0], nn.Linear)
        assert encoder.encoder[0].in_features == input_dim
        assert encoder.encoder[0].out_features == hidden_dim

        # 测试 forward 方法
        input_tensor = torch.randn(1, input_dim)
        output_tensor = encoder(input_tensor)
        assert output_tensor.shape == (1, hidden_dim)

In [8]:
test_AgentEncoder_DefaultHiddenDim()
test_AgentEncoder_CustomHiddenDim()

# nn.Module
nn.Module 是所有神经网络模块的基类。它提供了构建和管理神经网络的基本功能。通过继承 nn.Module，您可以定义自己的神经网络层和模型。

主要作用
- 模块化设计:
允许将神经网络分解为可重用的模块或层。
每个模块可以包含子模块，形成复杂的网络结构。
- 参数管理:
自动管理模型的参数（权重和偏置）。
提供 parameters() 方法来获取所有参数，方便优化器进行更新。
- 前向传播:
需要实现 forward 方法来定义前向传播逻辑。
调用模型实例时，会自动调用 forward 方法。
- 设备管理:
提供 to(device) 方法将模型移动到指定的设备（如 CPU 或 GPU）。
提供 cuda() 和 cpu() 方法方便地将模型移动到 GPU 或 CPU。
- 保存和加载:
提供 state_dict() 方法获取模型的状态字典。
提供 load_state_dict(state_dict) 方法加载状态字典。

In [ ]:
import torch
import torch.nn as nn

class SimpleNet(nn.Module):
    def __init__(self):
        super(SimpleNet, self).__init__()
        self.fc1 = nn.Linear(784, 256)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(256, 10)

    def forward(self, x):
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        return x

net = SimpleNet()
print(net)

# nn.Sequential
nn.Sequential 是一个容器，用于按顺序排列多个模块。它简化了模型的定义，特别适用于简单的线性网络结构。

主要作用
- 简化模型定义:
可以按顺序添加多个模块，而不需要显式地定义 forward 方法。适用于简单的线性网络结构。
- 模块管理:
自动管理添加的模块。提供 parameters() 方法来获取所有参数。
- 前向传播:
自动按顺序执行添加的模块。不需要手动实现 forward 方法。

In [ ]:
import torch
import torch.nn as nn

net = nn.Sequential(
    nn.Linear(784, 256),
    nn.ReLU(),
    nn.Linear(256, 10)
)

print(net)

# MAVAC test


根据提供的代码片段，`MAVACSota` 类是用于多智能体强化学习（MARL）的模型，继承自 `nn.Module`。以下是对其各个参数的详细解释：

### 初始化参数 (`__init__` 方法)

```python
def __init__(
    self,
    agent_obs_shape: Union[int, SequenceType],
    global_obs_shape: Union[int, SequenceType],
    action_shape: Union[int, SequenceType],
    agent_num: int,
    actor_head_hidden_size: int = 256,
    actor_head_layer_num: int = 2,
    critic_head_hidden_size: int = 512,
    critic_head_layer_num: int = 1,
    action_space: str = 'discrete',
    activation: Optional[nn.Module] = nn.ReLU(),
    norm_type: Optional[str] = None,
    sigma_type: Optional[str] = 'independent',
    bound_type: Optional[str] = None,
) -> None:
```

#### 参数详解

1. **agent_obs_shape**:
   - 类型：`Union[int, SequenceType]`
   - 描述：单个智能体的观测空间形状。可以是一个整数（表示一维观测）或一个序列（表示多维观测）。
   - 作用：用于定义智能体观测的输入维度。

2. **global_obs_shape**:
   - 类型：`Union[int, SequenceType]`
   - 描述：全局观测空间形状。可以是一个整数（表示一维观测）或一个序列（表示多维观测）。
   - 作用：用于定义全局观测的输入维度。

3. **action_shape**:
   - 类型：`Union[int, SequenceType]`
   - 描述：动作空间形状。可以是一个整数（表示离散动作）或一个序列（表示连续动作）。
   - 作用：用于定义动作输出的维度。

4. **agent_num**:
   - 类型：`int`
   - 描述：智能体的数量。
   - 作用：用于确定参与任务的智能体数量。

5. **actor_head_hidden_size**:
   - 类型：`int`
   - 默认值：`256`
   - 描述：Actor 网络头部隐藏层的大小。
   - 作用：控制 Actor 网络中全连接层的神经元数量。

6. **actor_head_layer_num**:
   - 类型：`int`
   - 默认值：`2`
   - 描述：Actor 网络头部层数。
   - 作用：确定 Actor 网络中全连接层的数量。

7. **critic_head_hidden_size**:
   - 类型：`int`
   - 默认值：`512`
   - 描述：Critic 网络头部隐藏层的大小。
   - 作用：控制 Critic 网络中全连接层的神经元数量。

8. **critic_head_layer_num**:
   - 类型：`int`
   - 默认值：`1`
   - 描述：Critic 网络头部层数。
   - 作用：确定 Critic 网络中全连接层的数量。

9. **action_space**:
   - 类型：`str`
   - 默认值：`'discrete'`
   - 描述：动作空间类型，可以是 `'discrete'` 或 `'continuous'`。
   - 作用：确定动作空间是离散还是连续，影响后续网络结构的选择。

10. **activation**:
    - 类型：`Optional[nn.Module]`
    - 默认值：`nn.ReLU()`
    - 描述：激活函数，默认使用 ReLU 激活函数。
    - 作用：应用于网络中的非线性变换。

11. **norm_type**:
    - 类型：`Optional[str]`
    - 默认值：`None`
    - 描述：归一化类型，见 `ding.torch_utils.fc_block` 的更多细节。
    - 作用：用于在网络中添加归一化层，如 BatchNorm、LayerNorm 等。

12. **sigma_type**:
    - 类型：`Optional[str]`
    - 默认值：`'independent'`
    - 描述：在连续动作空间中，用于确定标准差的生成方式。
    - 作用：影响连续动作分布的标准差计算方式。

13. **bound_type**:
    - 类型：`Optional[str]`
    - 默认值：`None`
    - 描述：在连续动作空间中，用于确定动作的边界处理方式。
    - 作用：确保生成的动作在合理的范围内。

### 总结

这些参数共同决定了 `MAVACSota` 模型的结构和行为，包括观测空间、动作空间、网络结构、激活函数等。通过调整这些参数，可以灵活地适应不同的多智能体强化学习任务和环境需求。

`@MODEL_REGISTRY.register('mavac_sota')` 的作用是将 `MAVACSota` 模型类注册到 `MODEL_REGISTRY` 这个全局注册表中，以便后续可以通过名称动态调用和实例化该模型。具体作用如下：

---

### 1. **核心功能**
- **注册模型**  
  通过装饰器语法，将 `MAVACSota` 类与字符串标识符 `'mavac_sota'` 绑定，并存入 `MODEL_REGISTRY`（一个 `Registry` 类的实例）。  
  等价于手动调用：
  ```python
  MODEL_REGISTRY.register(module_name='mavac_sota', module=MAVACSota)
  ```

- **实现解耦**  
  其他代码无需直接导入 `MAVACSota` 类，只需通过 `MODEL_REGISTRY.get('mavac_sota')` 即可获取模型类，实现模块间的解耦。

---

### 2. **Registry 类的作用**
- **统一管理模块**  
  `Registry` 继承自 `dict`，本质是一个字典，存储 `name -> module` 的映射。例如：
  ```python
  MODEL_REGISTRY = Registry()
  # 注册后，MODEL_REGISTRY 的内容变为：
  {'mavac_sota': MAVACSota}
  ```

- **动态构建对象**  
  通过 `build()` 方法，可以根据注册名动态实例化模型：
  ```python
  model = MODEL_REGISTRY.build('mavac_sota', agent_obs_shape=..., global_obs_shape=..., ...)
  ```

---

### 3. **实际应用场景**
- **配置化驱动**  
  在配置文件（如 YAML）中只需指定模型名 `mavac_sota`，代码即可自动加载对应模型：
  ```yaml
  model:
    type: mavac_sota
    args:
      agent_obs_shape: 64
      global_obs_shape: 128
      ...
  ```

- **灵活扩展**  
  新增模型时，只需用 `@MODEL_REGISTRY.register('new_model')` 装饰新类，无需修改其他代码即可集成。

---

### 4. **技术实现细节**
- **装饰器模式**  
  当 `@MODEL_REGISTRY.register('mavac_sota')` 装饰类时，会调用 `Registry.register()` 方法，将类对象存入字典。

- **模块元信息追踪**  
  若启用 `_DI_ENGINE_REG_TRACE_IS_ON`，会记录模型注册的代码位置（文件名和行号），便于调试：
  ```python
  # 查询注册信息
  print(MODEL_REGISTRY.query_details())
  # 输出: OrderedDict([('mavac_sota', ('path/to/file.py', 123))])
  ```

---

### 5. **与代码的关联**
在 `MAVACSota` 的定义中：
```python
@MODEL_REGISTRY.register('mavac_sota')
class MAVACSota(nn.Module):
    ...
```
- **`MODEL_REGISTRY`** 是一个全局注册表实例。
- **`register`** 方法将类名 `'mavac_sota'` 和类对象 `MAVACSota` 绑定。

---

### 总结
通过 `@MODEL_REGISTRY.register('mavac_sota')`，模型 `MAVACSota` 被注册到一个中央仓库中，后续可以通过字符串 `'mavac_sota'` 动态获取并实例化该模型。这种设计模式广泛应用于深度学习框架（如 Detectron2、MMDetection），以实现高度模块化和可配置化。

In [46]:
import torch
import unittest
from ding.model.template import MAVACSota
from ding.torch_utils import to_tensor


In [57]:
class TestMAVACSota(unittest.TestCase):
    def setUp(self):
        self.agent_obs_shape = 174
        self.global_obs_shape = 442
        self.action_shape = 4
        self.agent_num = 3
        self.action_space = 'discrete'

    def test_compute_actor_discrete(self):
        model = MAVACSota(
            self.agent_obs_shape, self.global_obs_shape, self.action_shape, self.agent_num, action_space='discrete'
        )
        inputs = {
            'agent_state': torch.randn(4, self.agent_obs_shape),
            'action_mask': torch.ones(4, self.action_shape)
        }
        actor_outputs = model(inputs, 'compute_actor')
        print("actor_outputs shape ", actor_outputs['logit'].shape)
        self.assertEqual(actor_outputs['logit'].shape, torch.Size([4, self.action_shape]))

    def test_compute_critic_sota(self):
        model = MAVACSota(
            self.agent_obs_shape, self.global_obs_shape, self.action_shape, self.agent_num,
        )
        inputs = {
            'agent_state': torch.randn(4, self.agent_obs_shape),
            'cls_state': torch.randn(4, self.agent_obs_shape),
            'global_state': torch.randn(4, self.global_obs_shape)
        }
        critic_outputs = model(inputs, 'compute_critic')
        self.assertEqual(critic_outputs['value'].shape, torch.Size([4]))

    def test_compute_critic_non_sota(self):
        model = MAVACSota(
            self.agent_obs_shape, self.global_obs_shape, self.action_shape, self.agent_num,
        )
        inputs = {
            'agent_state': torch.randn(4, self.agent_obs_shape),
            'cls_state': torch.randn(4, self.agent_obs_shape),
            'global_state': torch.randn(4, self.global_obs_shape)
        }
        critic_outputs = model(inputs, 'compute_critic')
        self.assertEqual(critic_outputs['value'].shape, torch.Size([4]))

    def test_compute_actor_critic(self):
        model = MAVACSota(
            self.agent_obs_shape, self.global_obs_shape, self.action_shape, self.agent_num, action_space='discrete'
        )
        inputs = {
            'agent_state': torch.randn(4, self.agent_obs_shape),
            'cls_state': torch.randn(4, self.agent_obs_shape),
            'global_state': torch.randn(4, self.global_obs_shape),
            'action_mask': torch.ones(4, self.action_shape)
        }
        outputs = model(inputs, 'compute_actor_critic')
        self.assertEqual(outputs['logit'].shape, torch.Size([4, self.action_shape]))
        self.assertEqual(outputs['value'].shape, torch.Size([4]))


In [58]:
mavac_sota_test = TestMAVACSota()
mavac_sota_test.setUp()
mavac_sota_test.test_compute_critic_sota()

ValueError: not enough values to unpack (expected 3, got 2)

torch.optim.Adam 是 PyTorch 中实现 Adam（Adaptive Moment Estimation）优化算法的优化器类。
Adam 是一种常用的自适应学习率优化算法，结合了 AdaGrad 和 RMSProp 算法的优点，能够自适应地调整每个参数的学习率。
- 优点
自适应学习率：能够根据每个参数的梯度情况自动调整学习率，对于不同的参数可以有不同的更新步长。
处理稀疏梯度：在处理稀疏数据时表现良好，因为它能够自适应地调整学习率，使得对于稀疏特征的更新不会过于激进。
收敛速度快：通常比传统的随机梯度下降（SGD）算法收敛速度更快，能够更快地找到最优解。
## 原理
Adam 算法通过计算梯度的一阶矩估计（均值）和二阶矩估计（方差）来为不同的参数动态调整学习率。具体步骤如下：

In [69]:
import torch
import torch.nn as nn
import torch.optim as optim
import unittest

# 定义一个简单的线性回归模型
class LinearRegression(nn.Module):
    def __init__(self):
        super(LinearRegression, self).__init__()
        self.linear = nn.Linear(1, 1)  # 输入维度为 1，输出维度为 1

    def forward(self, x):
        return self.linear(x)

class TestAdamOptimizer(unittest.TestCase):
    def test_adam_optimizer(self):
        # 生成一些简单的训练数据
        x = torch.tensor([[1.0], [2.0], [3.0], [4.0]], dtype=torch.float32)
        y = torch.tensor([[2.0], [4.0], [6.0], [8.0]], dtype=torch.float32)

        # 初始化模型
        model = LinearRegression()

        # 定义损失函数和优化器
        criterion = nn.MSELoss()
        optimizer = optim.Adam(model.parameters(), lr=0.01)

        # 训练模型
        num_epochs = 3000
        for epoch in range(num_epochs):
            # 前向传播
            outputs = model(x)
            loss = criterion(outputs, y)

            # 反向传播和优化
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            if (epoch + 1) % 100 == 0:
                print(f'Epoch [{epoch + 1}/{num_epochs}], Loss: {loss.item():.4f}')

        # 测试模型
        with torch.no_grad():
            test_input = torch.tensor([[5.0]], dtype=torch.float32)
            predicted = model(test_input)
            print(f'Predicted value for input 5: {predicted.item():.4f}')

        # 断言损失值足够小
        self.assertTrue(loss.item() < 0.01)

# if __name__ == '__main__':
#     unittest.main()

In [70]:
adam_test = TestAdamOptimizer()
adam_test.test_adam_optimizer()

Epoch [100/3000], Loss: 16.0117
Epoch [200/3000], Loss: 3.7753
Epoch [300/3000], Loss: 1.1573
Epoch [400/3000], Loss: 0.7688
Epoch [500/3000], Loss: 0.6736
Epoch [600/3000], Loss: 0.5996
Epoch [700/3000], Loss: 0.5274
Epoch [800/3000], Loss: 0.4577
Epoch [900/3000], Loss: 0.3920
Epoch [1000/3000], Loss: 0.3311
Epoch [1100/3000], Loss: 0.2759
Epoch [1200/3000], Loss: 0.2266
Epoch [1300/3000], Loss: 0.1834
Epoch [1400/3000], Loss: 0.1463
Epoch [1500/3000], Loss: 0.1148
Epoch [1600/3000], Loss: 0.0887
Epoch [1700/3000], Loss: 0.0674
Epoch [1800/3000], Loss: 0.0503
Epoch [1900/3000], Loss: 0.0369
Epoch [2000/3000], Loss: 0.0265
Epoch [2100/3000], Loss: 0.0187
Epoch [2200/3000], Loss: 0.0129
Epoch [2300/3000], Loss: 0.0087
Epoch [2400/3000], Loss: 0.0058
Epoch [2500/3000], Loss: 0.0037
Epoch [2600/3000], Loss: 0.0023
Epoch [2700/3000], Loss: 0.0014
Epoch [2800/3000], Loss: 0.0009
Epoch [2900/3000], Loss: 0.0005
Epoch [3000/3000], Loss: 0.0003
Predicted value for input 5: 9.9696
